![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
<img src="img/columns.png" width=650>

### Terminology

- Database
- Schema
- Queries
- Tables
- Views
- Primary Keys
- Foreign Keys
- Indexes
- Functions & User Defined Functions (UDFs)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=650>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

***
## Goal 3: Get going with sqlite!

In [3]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [4]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [5]:
cursor.execute("SELECT name, type FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports', 'table'), ('airlines', 'table'), ('routes', 'table')]


In [6]:
cursor.description

(('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None))

In [7]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(*cursor.fetchall(), sep='\n')

('airports', 'table')
('ix_airports_index', 'index')
('airlines', 'table')
('ix_airlines_index', 'index')
('routes', 'table')
('ix_routes_index', 'index')


In [8]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [9]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [10]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [11]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [12]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [13]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [14]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [15]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [16]:
statement = """
-- Select the unique list of countries with airports
SELECT DISTINCT country
FROM airports
WHERE country LIKE '%G%'

"""
cursor.execute(statement)
cursor.fetchall()

[('Papua New Guinea',),
 ('Greenland',),
 ('Algeria',),
 ('Ghana',),
 ('Nigeria',),
 ('Niger',),
 ('Togo',),
 ('Belgium',),
 ('Germany',),
 ('United Kingdom',),
 ('Guernsey',),
 ('Luxembourg',),
 ('Congo (Brazzaville)',),
 ('Congo (Kinshasa)',),
 ('Equatorial Guinea',),
 ('Madagascar',),
 ('Angola',),
 ('Gabon',),
 ('Gambia',),
 ('Guinea-Bissau',),
 ('Senegal',),
 ('Guinea',),
 ('Egypt',),
 ('Uganda',),
 ('Bulgaria',),
 ('Greece',),
 ('Hungary',),
 ('Portugal',),
 ('Bosnia and Herzegovina',),
 ('Gibraltar',),
 ('Montenegro',),
 ('Guatemala',),
 ('Nicaragua',),
 ('Tonga',),
 ('Afghanistan',),
 ('Argentina',),
 ('Guam',),
 ('Paraguay',),
 ('French Guiana',),
 ('Uruguay',),
 ('Guyana',),
 ('Antigua and Barbuda',),
 ('Guadeloupe',),
 ('Grenada',),
 ('Virgin Islands',),
 ('Anguilla',),
 ('Trinidad and Tobago',),
 ('British Virgin Islands',),
 ('Saint Vincent and the Grenadines',),
 ('Kyrgyzstan',),
 ('Bangladesh',),
 ('Hong Kong',),
 ('Singapore',),
 ('Mongolia',),
 ('Georgia',),
 ('Cocos (

In [17]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [18]:
cursor.execute("""
SELECT DISTINCT a.code, latitude, longitude
FROM airports a
JOIN routes r
    ON a.id = r.source_id
""")
print(*cursor.fetchall(), sep='\n')

('GKA', '-6.081689', '145.391881')
('MAG', '-5.207083', '145.7887')
('HGU', '-5.826789', '144.295861')
('LAE', '-6.569828', '146.726242')
('POM', '-9.443383', '147.22005')
('WWK', '-3.583828', '143.669186')
('UAK', '61.160517', '-45.425978')
('GOH', '64.190922', '-51.678064')
('SFJ', '67.016969', '-50.689325')
('THU', '76.531203', '-68.703161')
('AEY', '65.659994', '-18.072703')
('EGS', '65.283333', '-14.401389')
('IFJ', '66.058056', '-23.135278')
('KEF', '63.985', '-22.605556')
('RKV', '64.13', '-21.940556')
('YAM', '46.485001', '-84.509445')
('YAY', '51.391944', '-56.083056')
('YBC', '49.1325', '-68.204444')
('YBG', '48.330555', '-70.996391')
('YBK', '64.298889', '-96.077778')
('YBL', '49.950832', '-125.270833')
('YBR', '49.91', '-99.951944')
('YCB', '69.108055', '-105.138333')
('YCD', '49.052333', '-123.870167')
('YCG', '49.296389', '-117.6325')
('YCL', '47.990833', '-66.330278')
('YCO', '67.816667', '-115.143889')
('YCY', '70.486111', '-68.516667')
('YZS', '64.193333', '-83.359444'

('PBL', '10.4805', '-68.073025')
('PZO', '8.288528', '-62.760361')
('SVZ', '7.840831', '-72.439742')
('STD', '7.565111', '-72.035125')
('SOM', '8.945147', '-64.151083')
('VLN', '10.149733', '-67.9284')
('VLV', '9.340797', '-70.584089')
('ANU', '17.136749', '-61.792667')
('BGI', '13.074603', '-59.492456')
('DOM', '15.547028', '-61.3')
('FDF', '14.591033', '-61.003175')
('SFG', '18.099914', '-63.047197')
('PTP', '16.265306', '-61.531806')
('GND', '12.004247', '-61.786192')
('STT', '18.337306', '-64.973361')
('STX', '17.701889', '-64.798556')
('BQN', '18.494861', '-67.129444')
('MAZ', '18.255694', '-67.148472')
('PSE', '18.008306', '-66.563028')
('SJU', '18.439417', '-66.001833')
('SKB', '17.311194', '-62.718667')
('SLU', '14.020228', '-60.992936')
('UVF', '13.733194', '-60.952597')
('AUA', '12.501389', '-70.015221')
('BON', '12.131044', '-68.268511')
('CUR', '12.188853', '-68.959803')
('EUX', '17.496492', '-62.979439')
('SXM', '18.040953', '-63.1089')
('AXA', '18.204834', '-63.055084')
(

('RNL', '-11.5339', '160.063')
('GZO', '-8.09778', '156.864')
('MUA', '-8.32797', '157.263')
('SCZ', '-10.7203', '165.795')
('IRA', '-10.4497', '161.898')
('FRE', '-8.1075', '159.577')
('ATD', '-8.87333', '161.011')
('IKS', '71.6977', '128.903')
('CYX', '68.7406', '161.338')
('CKH', '70.6231', '147.902')
('KSN', '53.206944', '63.550278')
('DZN', '47.708333', '67.733333')
('KOV', '53.3291', '69.5946')
('VIJ', '18.4464', '-64.4275')
('NEV', '17.205678', '-62.589869')
('VIG', '8.6241', '-71.672819')
('SRA', '-27.9067', '-54.5204')
('HUU', '-9.878811', '-76.204797')
('RIB', '-11', '-66')
('GYA', '-10.8206', '-65.3456')
('PDA', '3.85', '-67.91')
('NQU', '5.7', '-77.28')
('LPD', '-1.33', '-69.58')
('LQM', '-0.18', '-74.77')
('LOH', '-3.99589', '-79.3719')
('SCY', '-0.910206', '-89.61745')
('VDC', '-14.862761', '-40.863106')
('MII', '-22.196892', '-49.9264')
('MEA', '-22.343', '-41.766')
('RXS', '11.597669', '122.751669')
('CYP', '12.072706', '124.545092')
('VRC', '13.576439', '124.205672')
(

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [19]:
# Which countries have the highest amount of active airlines?
cursor.execute("""
-- your code here;
SELECT country, count(*) AS airport_count
FROM airlines
WHERE active='Y'
GROUP BY country
ORDER by airport_count DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)


In [20]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
-- 
""")
print(*cursor.fetchall(), sep='\n')

In [21]:
# What about airports by timezones?
cursor.execute("""
-- your code here;""")
print(*cursor.fetchall(), sep='\n')

In [22]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [23]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [24]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
24,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
294,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
382,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
1007,A681062,Max,06/12/2014 02:22:00 PM,06/12/2014 02:22:00 PM,06/11/2011,Return to Owner,NaN,Dog,Neutered Male,3 years,Jack Russell Terrier Mix,Brown/White
1049,A727715,Max,10/07/2016 05:40:00 PM,10/07/2016 05:40:00 PM,05/25/2015,Adoption,NaN,Dog,Neutered Male,1 year,German Shepherd Mix,Tan/Black
...,...,...,...,...,...,...,...,...,...,...,...,...
111280,A773678,Max,06/14/2018 02:30:00 PM,06/14/2018 02:30:00 PM,01/04/2018,Adoption,NaN,Dog,Neutered Male,5 months,Black/Tan Hound Mix,Brown/Black
111643,A785678,Max,12/10/2018 10:15:00 PM,12/10/2018 10:15:00 PM,02/07/2018,Return to Owner,NaN,Dog,Intact Male,10 months,German Shepherd Mix,Brown/Black
111655,A777049,Max,08/02/2018 12:23:00 PM,08/02/2018 12:23:00 PM,07/22/2017,Return to Owner,NaN,Dog,Intact Male,1 year,Siberian Husky Mix,White
112245,A769530,Max,04/21/2018 07:19:00 PM,04/21/2018 07:19:00 PM,04/06/2017,Adoption,NaN,Dog,Neutered Male,1 year,Miniature Poodle Mix,White/Cream


In [25]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
10,A740079,Terry,12/26/2016 01:45:00 PM,12/26/2016 01:45:00 PM,12/10/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Australian Cattle Dog Mix,Black/White
12,A725530,King,06/09/2016 02:50:00 PM,06/09/2016 02:50:00 PM,10/30/2015,Adoption,NaN,Dog,Neutered Male,7 months,Pit Bull Mix,Brown Brindle/White
13,A722378,Canine,03/31/2016 07:08:00 PM,03/31/2016 07:08:00 PM,03/15/2010,Return to Owner,NaN,Dog,Neutered Male,6 years,German Shepherd Mix,Brown/Black
15,A681449,Buster,07/02/2014 07:01:00 PM,07/02/2014 07:01:00 PM,06/16/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Dogo Argentino Mix,White/Brown


## Goal 6: Transfering from sqlite to pandas

In [26]:
# pd_con = sqlite3.connect("flights.db")
# df = pd.read_sql_query("select * from airports limit 5;", pd_con)
# df

DatabaseError: Execution failed on sql 'select * from airports limit 5;': no such table: airports

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [31]:
# Which countries have the highest amount of active airports?
# df = pd.read_sql_query("""
# -- your code here;
# """, pd_con)
# df

In [32]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [33]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [40]:
statement = """

SELECT tr.Name, joined.Name, tr.UnitPrice
    FROM Track tr
    INNER JOIN (
        SELECT al.AlbumId, ar.Name
            FROM Album al
            INNER JOIN Artist ar ON al.ArtistId = ar.ArtistId
        )
    joined ON tr.AlbumId = joined.AlbumID
    WHERE tr.Unit
    

"""

cur.execute(statement)
print(*cur.fetchall(), sep='\n')

('For Those About To Rock (We Salute You)', 'AC/DC', 0.99)
('Balls to the Wall', 'Accept', 0.99)
('Fast As a Shark', 'Accept', 0.99)
('Restless and Wild', 'Accept', 0.99)
('Princess of the Dawn', 'Accept', 0.99)
('Put The Finger On You', 'AC/DC', 0.99)
("Let's Get It Up", 'AC/DC', 0.99)
('Inject The Venom', 'AC/DC', 0.99)
('Snowballed', 'AC/DC', 0.99)
('Evil Walks', 'AC/DC', 0.99)
('C.O.D.', 'AC/DC', 0.99)
('Breaking The Rules', 'AC/DC', 0.99)
('Night Of The Long Knives', 'AC/DC', 0.99)
('Spellbound', 'AC/DC', 0.99)
('Go Down', 'AC/DC', 0.99)
('Dog Eat Dog', 'AC/DC', 0.99)
('Let There Be Rock', 'AC/DC', 0.99)
('Bad Boy Boogie', 'AC/DC', 0.99)
('Problem Child', 'AC/DC', 0.99)
('Overdose', 'AC/DC', 0.99)
("Hell Ain't A Bad Place To Be", 'AC/DC', 0.99)
('Whole Lotta Rosie', 'AC/DC', 0.99)
('Walk On Water', 'Aerosmith', 0.99)
('Love In An Elevator', 'Aerosmith', 0.99)
('Rag Doll', 'Aerosmith', 0.99)
('What It Takes', 'Aerosmith', 0.99)
('Dude (Looks Like A Lady)', 'Aerosmith', 0.99)
("Jani

('Love Of My Life', 'Santana', 0.99)
('Put Your Lights On', 'Santana', 0.99)
('Africa Bamba', 'Santana', 0.99)
('Smooth', 'Santana', 0.99)
('Do You Like The Way', 'Santana', 0.99)
('Maria Maria', 'Santana', 0.99)
('Migra', 'Santana', 0.99)
('Corazon Espinado', 'Santana', 0.99)
('Wishing It Was', 'Santana', 0.99)
('El Farol', 'Santana', 0.99)
('Primavera', 'Santana', 0.99)
('The Calling', 'Santana', 0.99)
('Solução', 'Ed Motta', 0.99)
('Manuel', 'Ed Motta', 0.99)
('Entre E Ouça', 'Ed Motta', 0.99)
('Um Contrato Com Deus', 'Ed Motta', 0.99)
('Um Jantar Pra Dois', 'Ed Motta', 0.99)
('Vamos Dançar', 'Ed Motta', 0.99)
('Um Love', 'Ed Motta', 0.99)
('Seis Da Tarde', 'Ed Motta', 0.99)
('Baixo Rio', 'Ed Motta', 0.99)
('Sombras Do Meu Destino', 'Ed Motta', 0.99)
('Do You Have Other Loves?', 'Ed Motta', 0.99)
('Agora Que O Dia Acordou', 'Ed Motta', 0.99)
('Já!!!', 'Ed Motta', 0.99)
('A Rua', 'Ed Motta', 0.99)
("Now's The Time", 'Miles Davis', 0.99)
('Jeru', 'Miles Davis', 0.99)
('Compulsion', 'M

('Aviso Aos Navegantes', 'Lulu Santos', 0.99)
('Casa', 'Lulu Santos', 0.99)
('Condição', 'Lulu Santos', 0.99)
('Hyperconectividade', 'Lulu Santos', 0.99)
('O Descobridor Dos Sete Mares', 'Lulu Santos', 0.99)
('Satisfação', 'Lulu Santos', 0.99)
('Brumário', 'Lulu Santos', 0.99)
('Um Certo Alguém', 'Lulu Santos', 0.99)
('Fullgás', 'Lulu Santos', 0.99)
('Sábado À Noite', 'Lulu Santos', 0.99)
('A Cura', 'Lulu Santos', 0.99)
('Aquilo', 'Lulu Santos', 0.99)
('Atrás Do Trio Elétrico', 'Lulu Santos', 0.99)
('Senta A Pua', 'Lulu Santos', 0.99)
('Ro-Que-Se-Da-Ne', 'Lulu Santos', 0.99)
('Tudo Bem', 'Lulu Santos', 0.99)
('Toda Forma De Amor', 'Lulu Santos', 0.99)
('Tudo Igual', 'Lulu Santos', 0.99)
('Fogo De Palha', 'Lulu Santos', 0.99)
('Sereia', 'Lulu Santos', 0.99)
('Assaltaram A Gramática', 'Lulu Santos', 0.99)
('Se Você Pensa', 'Lulu Santos', 0.99)
('Lá Vem O Sol (Here Comes The Sun)', 'Lulu Santos', 0.99)
('O Último Romântico (Ao Vivo)', 'Lulu Santos', 0.99)
('Pseudo Silk Kimono', 'Marillion

('Compadre', 'Tim Maia', 0.99)
('Over Again', 'Tim Maia', 0.99)
('Réu Confesso', 'Tim Maia', 0.99)
('O Que Me Importa', 'Tim Maia', 0.99)
('Gostava Tanto De Você', 'Tim Maia', 0.99)
('Você', 'Tim Maia', 0.99)
('Não Quero Dinheiro', 'Tim Maia', 0.99)
('Eu Amo Você', 'Tim Maia', 0.99)
('A Festa Do Santo Reis', 'Tim Maia', 0.99)
("I Don't Know What To Do With Myself", 'Tim Maia', 0.99)
('Padre Cícero', 'Tim Maia', 0.99)
('Nosso Adeus', 'Tim Maia', 0.99)
('Canário Do Reino', 'Tim Maia', 0.99)
('Preciso Ser Amado', 'Tim Maia', 0.99)
('Balanço', 'Tim Maia', 0.99)
('Preciso Aprender A Ser Só', 'Tim Maia', 0.99)
('Esta É A Canção', 'Tim Maia', 0.99)
('Formigueiro', 'Tim Maia', 0.99)
('Comida', 'Titãs', 0.99)
('Go Back', 'Titãs', 0.99)
('Prá Dizer Adeus', 'Titãs', 0.99)
('Família', 'Titãs', 0.99)
('Os Cegos Do Castelo', 'Titãs', 0.99)
('O Pulso', 'Titãs', 0.99)
('Marvin', 'Titãs', 0.99)
('Nem 5 Minutos Guardados', 'Titãs', 0.99)
('Flores', 'Titãs', 0.99)
('Palavras', 'Titãs', 0.99)
('Hereditári

In [ ]:
# How many tables are in the database?

# your code here

In [ ]:
# What's the primary key of each table?

# your code here

In [ ]:
# What foreign keys join the tables together?

# your code here
# hint: use "PRAGMA foreign_key_list()"


In [ ]:
# What are the max and min dates in the Invoice table?
cur.execute("""
-- your code here;
""").fetchall()

In [ ]:
# What tables would you need to answer "what is your most popular track?"


In [ ]:
# What values from each table?


In [ ]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

In [ ]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

## Reflection

## Learning goals review:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

Please fill out today's [Exit Ticket](https://docs.google.com/forms/d/1O0dMGqjIV6-JIDYSIeGqTjIdep1BFiIVCc29UbEgwUM/edit)!